In [35]:
from openpyxl import load_workbook
import os
from functools import partial 
import pandas as pd
import json
from collections import OrderedDict
from docx import Document

In [2]:
class AbstractParserFichas:
    
        
    def get_files(self, path_dados = None):
        
        if path_dados is None:
            path_dados = self.path_dados
        
        files = [os.path.join(path_dados, f) for f
            in os.listdir(path_dados) if 'xls' in f.split('.')[-1]] 
        
        return files
    
    def wb_generator(self, path_dados):
        
        files = self.get_files(path_dados)
        
        for file in files:
            yield file, load_workbook(file, read_only=True, data_only=True)
    
    
    def get_sheet_by_name(self, wb, sheet_name_padrao):
    
        for sheet_name in wb.sheetnames:
            if sheet_name_padrao in sheet_name.lower():
                return wb[sheet_name]
        else:
            raise RuntimeError(f'Sheetname {sheet_name_padrao} nao encontrado na planilha!')
            
    def extract_with_col_mapper(self, wb, mapper):
    
        sheet = self.get_sheet_by_name(wb, mapper['sheet_name'])

        data = []

        for i in range(mapper['line_ini'], mapper['line_end']+1):
            parsed = {}
            for name, col in mapper['data_cells'].items():
                cell = col + str(i)
                parsed[name] = sheet[cell].value
            data.append(parsed)

        return data
    
    
    def extract_with_col_mapper_variable_rows(self, wb, mapper):
    
        sheet = self.get_sheet_by_name(wb, mapper['sheet_name'])

        data = []
        i = mapper['line_ini']
        while True:
            parsed = {}
            for name, col in mapper['data_cells'].items():
                cell = col + str(i)
                parsed[name] = sheet[cell].value
            if parsed[mapper['col_stop']] is None:
                break
            data.append(parsed)
            i+=1


        return data
    
    
    def extract_with_cell_mapper(self, wb, mapper):
    
        sheet = self.get_sheet_by_name(wb, mapper['sheet_name'])

        parsed = {}

        for name, cell in mapper['data_cells'].items():
            parsed[name] = sheet[cell].value

        return parsed
    
    def extract_whole_sheet(self, file, sheet_name):
    
        return pd.read_excel(file, sheet_name = sheet_name, thousands = ',').to_dict(orient = 'records')
    
    def salvar_file(self, parsed_ficha, path_salvar = None):
        
        f_name = os.path.split(parsed_ficha['file_original'])[-1].replace('.xlsx', '.json')
        
        if path_salvar is None:
            path_salvar = self.path_salvar
        
        if not os.path.exists(path_salvar):
            os.mkdir(path_salvar)
        
        path_file = os.path.join(path_salvar, f_name)
        
        with open(path_file, 'w', encoding='cp1252') as f:
            json.dump(parsed_ficha, f, ensure_ascii=False)
        
    
    
class ParserFichas(AbstractParserFichas):
    
    mapper_ficha_tecnica = dict(
        sheet_name = 'ficha técnica meta',
        data_cells = dict(
            secretaria = 'c4',
            numero_meta = 'c5',
            desc_meta = 'c6',
            indicador = 'c9',
            contexto = 'c10',
            info_compl = 'c11',
            valor_base = 'c12',
            serie_historica = 'c13',
            frequencia = 'c14',
            periodo = 'c15',
            forma_apuracao = 'c18',
            observacoes = 'c20',
            prev_21 = 'c23',
            prev_22 = 'c24',
            prev_23 = 'c25',
            prev_24 = 'c26',
            impacto_covid = 'b29'
        )
    )
    
    
    mapper_iniciativas = dict(
        sheet_name = 'iniciativas',
        line_ini = 4,
        col_stop = 'descricao',
        data_cells = {
            'id' : 'b',
            'descricao' : 'c',
            'orgao_unidade' : 'd',
            'detalhamento' : 'e',
            'valor_global' : 'f'
        }
    )
    
    
    mapper_alteracoes = dict(
        sheet_name = 'proposta de alteração',
        line_ini = 4,
        col_stop = 'meta_ou_iniciativa',
        data_cells = {
            'meta_ou_iniciativa' : 'b',
            'item' : 'c',
            'versao_publicada' : 'd',
            'proposta_alteracao' : 'e',
            'justificativa' : 'f'
        }
    )
    
    mapper_regionalizacao = dict(
        sheet_name = 'regionalização',
        line_ini = 3,
        col_stop = 'projecao_quadrienio',
        data_cells = {
            'subprefeitura' : 'b',
            'projecao_quadrienio' : 'c',
            'status_regionalizacao' : 'd'
        }
    )
    
    
    
    def __init__(self, path_dados, path_salvar):
        
        self.path_dados = path_dados
        self.files = self.get_files(path_dados)
        self.path_salvar = path_salvar
        self.set_methods()
    
    def set_methods(self):
        
        self.extract_ficha_tecnica = partial(self.extract_with_cell_mapper, 
                                                   mapper = self.mapper_ficha_tecnica)
        self.extract_iniciativas = partial(self.extract_with_col_mapper_variable_rows,
                                                 mapper= self.mapper_iniciativas)
        self.extract_alteracoes = partial(self.extract_with_col_mapper_variable_rows, 
                                                mapper = self.mapper_alteracoes)
        self.extract_regionalizacao = partial(self.extract_with_col_mapper_variable_rows, 
                                                    mapper = self.mapper_regionalizacao)
        self.extract_custos = partial(self.extract_whole_sheet, sheet_name = 'Custo')
        self.extract_orcamento = partial(self.extract_whole_sheet, sheet_name = 'Orçamento')
        
    def extract_ficha(self, file, wb = None, salvar = False, path_salvar = None):
        
        if wb is None:
            wb = load_workbook(file)
            
        parsed = {
            'ficha_tecnica' : self.extract_ficha_tecnica(wb),
            'iniciativas' : self.extract_iniciativas(wb),
            'alteracoes' : self.extract_alteracoes(wb),
            'regionalizacao' : self.extract_regionalizacao(wb),
            'custos' : self.extract_custos(file),
            'orcamento' : self.extract_orcamento(file),
            'file_original' : file
            }
        
        if salvar:
            self.salvar_file(parsed, path_salvar)
            
        return parsed
            
    def extract_all_fichas(self, path_dados = None, salvar = True, path_salvar = None):
        
        if path_dados is None:
            path_dados = self.path_dados
        parsed_data = []
        wbs = self.wb_generator(path_dados)
        for file, wb in wbs:
            
            ficha = self.extract_ficha(file, wb, salvar, path_salvar)
            parsed_data.append(ficha)
        
        return parsed_data
            
    def __call__(self):
        
        return self.extract_all_fichas()
            

In [3]:
parser = ParserFichas('original_data/Devolutiva 11-jun', 'rodada_1')

In [4]:
fichas = parser()

In [55]:
class TableBuilder:
    
    def __init__(self, ficha, docx = None):
        
        self.ficha = ficha
        if docx is None:
            docx = Document()
        self.doc = docx
        
    def build_iniciativas(self, iniciativas):
    
        lista_textos = []
        for ini in iniciativas:
            texto = f"{ini['id']}) {ini['descricao']}"
            lista_textos.append(texto)
        return '\n'.join(lista_textos)
    
    def table_data(self, ficha = None):
        
        if ficha is None:
            ficha = self.ficha
        
        tabela_word = OrderedDict()
        
        #note que o primeiro tem espaço
        tabela_word['Meta '] = str(ficha['ficha_tecnica']['numero_meta'])
        tabela_word['Objetivo estratégico'] = ''
        tabela_word['Meta'] = ficha['ficha_tecnica']['desc_meta']
        tabela_word['Indicador'] = ficha['ficha_tecnica']['indicador']
        tabela_word['Informações Complementares'] = ficha['ficha_tecnica']['info_compl']
        tabela_word['ODS vinculados'] = ''
        tabela_word['Iniciativas'] = self.build_iniciativas(ficha['iniciativas'])
        tabela_word['Secretaria Responsável'] = ficha['ficha_tecnica']['secretaria']
        
        return tabela_word
    
    
    def build_table(self, table_data = None, docx = None):
        
        if docx is None:
            docx = self.doc
        if table_data is None:
            table_data = self.table_data(self.ficha)
            
        table = docx.add_table(rows=len(table_data), cols=2)
        
        i = 0
        for cell_nom, cell_value in table_data.items():
            row = table.rows[i]
            row.cells[0].text = cell_nom
            row.cells[1].text = cell_value
            i+=1
            
        table.style = 'LightList-Accent1'
        
        docx.add_page_break()
        
        return docx
        
    def __call__(self, table_data = None, docx = None):
        
        return self.build_table(table_data, docx)

In [56]:
ficha = fichas[0]

In [57]:
builder =  TableBuilder(ficha)

In [58]:
docxs = builder()

In [59]:
docxs.save('teste.docx')

In [14]:
ficha.keys()

dict_keys(['ficha_tecnica', 'iniciativas', 'alteracoes', 'regionalizacao', 'custos', 'orcamento', 'file_original'])

In [27]:
print(build_iniciativas(ficha['iniciativas']))

1.1) Monitoramento dos servidores responsáveis pelo controle interno
1.2) Capacitação dos servidores responsáveis pelo controle interno
2.1) Programa de Integridade e Boas Práticas implementado em 100% das Secretarias e Subprefeituras
2.2) Capacitação das equipes do Programa de Integridade e Boas Práticas 
2.3) Publicação dos Planos de Integridade
3) Ações de capacitação sobre transparência no setor público
4) Monitoramento de contratos emergenciais
5) Monitoramento de pregões
6) Monitoramento de cargos comissionados
7) Campanha de divulgação do Índice de Integridade
8.1) Fortalecimento da Auditoria Interna na PMSP, através da adoção da Metodologia IA-CM (Modelo de Capacidade de Auditoria Interna) na Coordenadoria de Auditoria Geral da CGM
8.2) Desdobramento da 8.1
8.3) Desdobramento da 8.1
8.4) Desdobramento da 8.1
8.5) Desdobramento da 8.1
8.6) Desdobramento da 8.1
8.7) Desdobramento da 8.1
8.8) Desdobramento da 8.1
9) Criação e entrega anual do Selo CGM de Transparência e Boas Práti

In [22]:
ficha['iniciativas']

[{'id': '1.1',
  'descricao': 'Monitoramento dos servidores responsáveis pelo controle interno',
  'orgao_unidade': 'CGM / AT',
  'detalhamento': 'A Controladoria Geral do Município (CGM) tem o objetivo de disseminar entre as Secretarias e Subprefeituras da Administração Pública diretrizes e orientações que visam tornar suas atividades mais eficientes, íntegras e transparentes. Para alcançar esse intuito, a partir de 2020, o Decreto n.º 59.496/2020 (art. 42) tornou obrigatória em todos os órgãos e entidades da PMSP, a indicação de servidor que será responsável por implementar em suas unidades procedimentos de controle planejados pela CGM.',
  'valor_global': None},
 {'id': '1.2',
  'descricao': 'Capacitação dos servidores responsáveis pelo controle interno',
  'orgao_unidade': 'CGM / AUDI / COPI / OGM',
  'detalhamento': 'A CGM oferecerá, anualmente, oficinas de capacitação voltadas aos responsáveis pelo controle interno, com o objetivo de orientá-los quanto a temas de controle, transp